# Depression detection using Audio (CNN)

In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import numpy as np
import pandas as pd 
import sklearn
import gc

Mounted at /content/drive


In [9]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import metrics
from sklearn.metrics import classification_report

In [10]:
def Thresholding(Y_pred, threshold):
  Y_pred_temp = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred_temp.append(0)
    else:
      Y_pred_temp.append(1)

  return np.array(Y_pred_temp)

def balanceDataset(X_train,Y_train):
  #X_train_d and X_train_nd represeting depressed and non-depressed input data points
  X_train_nd = X_train[Y_train==0]
  X_train_d = X_train[Y_train==1]

  Y_train_d = Y_train[Y_train==1]
  size = X_train_nd.shape[0] - X_train_d.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_d.shape[0]-1)
    added_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_d[index]
    X_added = X_train[added_index]

    Y_add = Y_train_d[index]
    Y_added = Y_train[added_index]

    X_train[added_index] = X_add
    X_train.append(X_added)

    Y_train[added_index] = Y_add
    Y_train.append(Y_added)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train

In [11]:

def checkAndRemoveFirst7columns(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    return X



dev = np.array(pd.read_csv('/content/drive/My Drive/diacwoz/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('/content/drive/My Drive/diacwoz/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
train = np.array(pd.read_csv('/content/drive/My Drive/diacwoz/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
X_train = []
Y_train = []

X_test = []
Y_test = []
size = 40000

for i in range(int(0.9 * len(train))):
    data = pd.read_csv('/content/drive/My Drive/diacwoz/train_data/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    data = data.iloc[data.shape[0]-size:,:].values
    data = checkAndRemoveFirst7columns(data)
    data = np.array(data, dtype = object)
    X_train.append(data)
    Y_train.append(train[i][1])
    del data
    gc.collect()

train = []
gc.collect()

for i in range(int(0.9 * len(dev))):
    data = pd.read_csv('/content/drive/My Drive/diacwoz/dev_data/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    data = data.iloc[data.shape[0]-size:,:].values
    data = checkAndRemoveFirst7columns(data)
    data = np.array(data, dtype = object)
    X_train.append(data)
    Y_train.append(dev[i][1])
    del data
    gc.collect()

dev = []
gc.collect()

X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.array(Y_train, dtype = object)
X_balanced, Y_balanced = balanceDataset(X_train,Y_train)
gc.collect()
print(X_balanced.shape)


(170, 40000, 74)


In [12]:
import tensorflow as tf
class CNN_audio:

  def __init__(self):
    classifier = Sequential()
    classifier.add(Conv1D(60, 10, input_shape = (40000, 74), activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
    classifier.add(Conv1D(30, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
    classifier.add(Conv1D(15, 5, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 3))
    classifier.add(Flatten())
    classifier.add(Dropout(0.8))
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )
    self.classifier = classifier
  
  def modelFit(self, X, Y, epoch = 10):
    self.classifier.fit(X, Y, epochs=epoch)

  def modelPredict(self, X):
    return self.classifier.predict(X)

model = CNN_audio()
Y_train = Y_train.astype(int)
model.modelFit(X_balanced, Y_balanced, 6)


Epoch 1/6
6/6 [==============================] - 32s 5s/step - loss: 35.4404 - accuracy: 0.5353
Epoch 2/6
6/6 [==============================] - 33s 5s/step - loss: 2.1109 - accuracy: 0.4941
Epoch 3/6
6/6 [==============================] - 29s 5s/step - loss: 0.6990 - accuracy: 0.5471
Epoch 4/6
6/6 [==============================] - 29s 5s/step - loss: 0.6738 - accuracy: 0.5647
Epoch 5/6
6/6 [==============================] - 30s 5s/step - loss: 0.6266 - accuracy: 0.6235
Epoch 6/6
6/6 [==============================] - 30s 5s/step - loss: 0.5464 - accuracy: 0.7529


In [13]:
gc.collect()

X_test = []
Y_test = []

for i in range(int(0.9 * len(test))):
    data = pd.read_csv('/content/drive/My Drive/diacwoz/test_data/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    data = data.iloc[data.shape[0]-size:,:].values
    data = checkAndRemoveFirst7columns(data)
    data = np.array(data, dtype = object)
    X_test.append(data)
    Y_test.append(test[i][1])


In [18]:
X_test = np.asarray(X_test).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.float32)
gc.collect()

Y_pred = model.modelPredict(X_test)
Y_pred = Thresholding(Y_pred, 0.4)
print(classification_report(Y_test,Y_pred))

2/2 [==============================] - 2s 474ms/step
Y_pred:  (42, 1)
              precision    recall  f1-score   support

         0.0       0.77      0.71      0.74        28
         1.0       0.50      0.57      0.53        14

    accuracy                           0.67        42
   macro avg       0.63      0.64      0.64        42
weighted avg       0.68      0.67      0.67        42

